**Bitcoin price prediction with Bayesian Neural Network Classification using torchBNN and PyTorch.**

In [51]:
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

import datetime
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchinfo import summary
import torchbnn as bnn

In [52]:
cwd = os.getcwd()
from os.path import dirname, abspath
while not cwd.endswith('BT4222_repo'):
    cwd = os.path.dirname(cwd)

In [53]:
PATH = os.path.join(cwd,'data','cooked_data','cooked_complete_dataset.csv')
df = pd.read_csv(PATH)
df.dropna(inplace = True)

df['date'] = df['date'].apply(lambda x: datetime.datetime.strptime(x, "%d/%m/%y"))
df["date"] = pd.to_datetime(df["date"], format='%d/%m/%Y', infer_datetime_format=True)
df.head()

,date,Adj_Close_BTC-USD,Open_BTC-USD,High_BTC-USD,Low_BTC-USD,Volume_BTC-USD,Adj_Close_SPY,Adj_Close_GLD,Adj_Close_CHFUSD=X,Adj_Close_CNYUSD=X,Adj_Close_EURUSD=X,Adj_Close_GBPUSD=X,Adj_Close_JPYUSD=X,coindesk_sentiment,num_of_coindesk_posts,reddit_comments_sentiments,top_50_reddit_posts_sentiments,blockchain_transactions_per_block,blockchain_hash_rates
0,2020-12-14,19246.6445,19144.4922,19305.0996,19012.7090,2.250000e+10,361.926788,171.539993,1.125442,0.152772,1.213340,1.331824,0.009621,0.249489,12,0.158060,0.677618,2167.93103,134533588
1,2020-12-15,19417.0762,19246.9199,19525.0078,19079.8418,2.670000e+10,366.819824,173.940002,1.127930,0.152679,1.214890,1.333084,0.009614,0.173773,18,0.101930,0.447277,2288.85714,133351912
2,2020-12-16,21310.5977,19418.8184,21458.9082,19298.3164,4.440000e+10,367.395508,174.899994,1.129382,0.152945,1.215430,1.344447,0.009649,0.341491,11,0.127344,0.480809,2204.31469,132323572
3,2020-12-17,22805.1621,21308.3516,23642.6602,21234.6758,7.140000e+10,369.449982,176.740006,1.129446,0.153109,1.219959,1.350293,0.009664,0.197572,10,0.135945,0.539729,2399.07752,132373209
4,2020-12-18,23137.9609,22806.7969,23238.6016,22399.8125,4.040000e+10,367.974793,176.440002,1.130301,0.153090,1.226272,1.357018,0.009696,0.315601,2,0.135441,0.449503,2392.03185,131791042


In [54]:
## Create binary label
df["class_y"] = df["Adj_Close_BTC-USD"].shift(1).dropna()
df["class_y"] = df.apply(lambda x : 1 if x["class_y"] < x["Adj_Close_BTC-USD"] else 0 , axis = 1)
df.head()

,date,Adj_Close_BTC-USD,Open_BTC-USD,High_BTC-USD,Low_BTC-USD,Volume_BTC-USD,Adj_Close_SPY,Adj_Close_GLD,Adj_Close_CHFUSD=X,Adj_Close_CNYUSD=X,Adj_Close_EURUSD=X,Adj_Close_GBPUSD=X,Adj_Close_JPYUSD=X,coindesk_sentiment,num_of_coindesk_posts,reddit_comments_sentiments,top_50_reddit_posts_sentiments,blockchain_transactions_per_block,blockchain_hash_rates,class_y
0,2020-12-14,19246.6445,19144.4922,19305.0996,19012.7090,2.250000e+10,361.926788,171.539993,1.125442,0.152772,1.213340,1.331824,0.009621,0.249489,12,0.158060,0.677618,2167.93103,134533588,0
1,2020-12-15,19417.0762,19246.9199,19525.0078,19079.8418,2.670000e+10,366.819824,173.940002,1.127930,0.152679,1.214890,1.333084,0.009614,0.173773,18,0.101930,0.447277,2288.85714,133351912,1
2,2020-12-16,21310.5977,19418.8184,21458.9082,19298.3164,4.440000e+10,367.395508,174.899994,1.129382,0.152945,1.215430,1.344447,0.009649,0.341491,11,0.127344,0.480809,2204.31469,132323572,1
3,2020-12-17,22805.1621,21308.3516,23642.6602,21234.6758,7.140000e+10,369.449982,176.740006,1.129446,0.153109,1.219959,1.350293,0.009664,0.197572,10,0.135945,0.539729,2399.07752,132373209,1
4,2020-12-18,23137.9609,22806.7969,23238.6016,22399.8125,4.040000e+10,367.974793,176.440002,1.130301,0.153090,1.226272,1.357018,0.009696,0.315601,2,0.135441,0.449503,2392.03185,131791042,1


**Helper Functions for creating lags and scaling.**

In [55]:
def lag(data, dic):
    cols = []
    for key, value in dic.items():
        for i in range(1, value+1):
            cols.append(data[key].shift(i).rename('{}_lag{}'.format(data[key].name, i)))
    return pd.concat([data["date"],data["class_y"]] + cols, axis = 1)

scaler = MinMaxScaler()

def scale_and_convert_to_tensor(Xtrain, Xtest, Ytrain, Ytest, scaleTarget = False):  
    global scaler
    
    # Standardise features
    Xtrain_standardised = scaler.fit_transform(Xtrain)
    Xtest_standardised = scaler.transform(Xtest)
    
    # Standardise target
    Ytrain_standardised = Ytrain
    Ytest_standardised = Ytest
    
    if scaleTarget:    
        Ytrain_standardised = scaler.fit_transform(np.array(Ytrain).reshape(-1, 1))
        Ytest_standardised = scaler.transform(np.array(Ytest).reshape(-1, 1))
    
    ## Change to tensor
    Xtrain_tensor = torch.from_numpy(Xtrain_standardised).float()
    Ytrain_tensor = torch.from_numpy(np.array(Ytrain_standardised)).float()
    Xtest_tensor = torch.from_numpy(Xtest_standardised).float()
    Ytest_tensor = torch.from_numpy(np.array(Ytest_standardised)).float()
        
    return (Xtrain_tensor, Xtest_tensor, Ytrain_tensor, Ytest_tensor)

## Without Sentiments

**1. Create feature lags**

In [56]:
feature_lags = {"Adj_Close_BTC-USD" : 1,
                "Adj_Close_SPY" : 1,
                "Adj_Close_GLD" : 1,
                "Adj_Close_CHFUSD=X" : 1,
                "Adj_Close_EURUSD=X" : 1,
                "Adj_Close_GBPUSD=X" : 1,
                "Adj_Close_JPYUSD=X" : 1,
                "blockchain_transactions_per_block" : 1,
                "blockchain_hash_rates" : 1}

data = lag(df, feature_lags)

**2. Handle train-test split**

In [57]:
data = data[(data["date"] >= "2021-01-01") & (data["date"] <= "2021-04-12")]

## train, validation and test split
train = data[(data["date"] >= "2021-01-01") & (data["date"] <= "2021-03-14")]
validation = data[(data["date"] >= "2021-03-15") & (data["date"] <= "2021-03-29")]
test = data[(data["date"] >= "2021-03-30") & (data["date"] <= "2021-04-12")]

## refit (refit = train + validation) and full for later use 
refit = data[(data["date"] >= "2021-01-01") & (data["date"] <= "2021-03-29")]
full = data.copy(deep = True)

## train 
X_train = train.drop(["date", "class_y"], axis = 1)
y_train = train["class_y"]

## val
X_val = validation.drop(["date", "class_y"], axis = 1)
y_val = validation["class_y"]

## test
X_test = test.drop(["date", "class_y"], axis = 1)
y_test = test["class_y"]

## refit
X_refit = refit.drop(["date", "class_y"], axis = 1)
y_refit = refit["class_y"]

## full
X_full = full.drop(["date", "class_y"], axis = 1)
y_full = full["class_y"]

**3. Standardise dataset and transform into tensors for pytorch**

In [58]:
## Standardise datasets and convert into tensors
Xtrain_tensor, Xval_tensor, Ytrain_tensor, Yval_tensor = scale_and_convert_to_tensor(X_train, X_val, y_train, y_val)
_, Xtest_tensor, _, Ytest_tensor = scale_and_convert_to_tensor(X_train, X_test, y_train, y_test)

print("X train tensor",Xtrain_tensor.shape)
print("Y train tensor",Ytrain_tensor.shape)

print("X val tensor",Xval_tensor.shape)
print("Y val tensor",Yval_tensor.shape)

print("X test tensor",Xtest_tensor.shape)
print("Y test tensor",Ytest_tensor.shape)

X train tensor torch.Size([73, 9])
Y train tensor torch.Size([73])
X val tensor torch.Size([15, 9])
Y val tensor torch.Size([15])
X test tensor torch.Size([14, 9])
Y test tensor torch.Size([14])


**4. Define BNN training and evaluation pipeline**

In [59]:
def train_model_and_evaluate_classification(Xtrain_tensor, Ytrain_tensor, Xtest_tensor, Ytest_tensor ,
                                            layers = [100,20], learning_param = 0.01, kl_weight = 0.01, steps = 100, threshold = 0.50, printStep = True):    
    in_features = Xtrain_tensor.shape[1]
    batch_size = Xtrain_tensor.shape[0]
    
    ## Ensure reproducibility
    seed = 1
    torch.manual_seed(seed)
    
    # Build model
    layer = []
    
    ## Input layer
    layer.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features = in_features, out_features = layers[0]))
    layer.append(nn.ReLU())
    
    ## Hidden layers
    for index, neurons in enumerate(layers):
        if index != (len(layers)-1):
            layer.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=neurons, out_features=layers[index+1]))
            layer.append(nn.ReLU())

    ## Output layer
    layer.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=layers[-1], out_features=1))
    layer.append(nn.Sigmoid())
    
    model = nn.Sequential(*layer)
    
    ### Define Loss - CrossEntropy for classification
    cross_entropy_loss = nn.BCELoss()
    kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)

    ## Define optimiser with learning rate
    optimizer = optim.Adam(model.parameters(), lr = learning_param)
    
    ### Train model
    for step in range(steps):
        torch.manual_seed(seed)
        
        pre = model(Xtrain_tensor)
        cross_entropy = cross_entropy_loss(pre, Ytrain_tensor.reshape(-1, 1).type(torch.FloatTensor))
        kl = kl_loss(model)
        total_cost = cross_entropy + kl_weight*kl

        optimizer.zero_grad()
        total_cost.backward()
        optimizer.step()
    
        if step%20==0 and printStep:
            print('[Step %d]: CE : %.8f, KL : %.8f' % (step , cross_entropy.item(), kl.item()))

    train_cross_entropy = cross_entropy.item()
    kl_loss = kl.item()
    
    torch.manual_seed(seed)
    ## Performance Evaluation on train
    y_predict_train = model(Xtrain_tensor)
    y_predict_binary_train = pd.Series(y_predict_train.detach().numpy().flatten()).apply(lambda x: 1 if x >= threshold else 0)
    train_accuracy = accuracy_score(Ytrain_tensor.detach().numpy(), y_predict_binary_train)
    
    ## Performance Evaluation on test
    y_predict = model(Xtest_tensor)
    y_predict_binary = pd.Series(y_predict.detach().numpy().flatten()).apply(lambda x: 1 if x >= threshold else 0)
    test_cross_entropy = cross_entropy_loss(y_predict, Ytest_tensor.reshape(-1, 1).type(torch.FloatTensor)).item()
    test_accuracy = accuracy_score(Ytest_tensor.detach().numpy(), y_predict_binary)

    return (y_predict_binary , train_cross_entropy, test_cross_entropy, train_accuracy, test_accuracy, kl_loss)


**5. Perform Grid Search**

Perform grid search and evaluate based on validation set.

In [60]:
### Hyperparameter Tuning --> find best parameters
train_accuracy_list = []
val_accuracy_list = []
combination = []

learning_param_list = pd.Series(np.linspace(0.001,0.5,50)).apply(lambda x: round(x,3))
kl_weight =  pd.Series(np.linspace(0.001,0.5,50)).apply(lambda x: round(x,3))
layers_list = ([32,16],[32,8],[32,16,8])

for layer in layers_list:
    print("--- Layer: ", layer)
    for lr in learning_param_list:
        print("-- Learning Param: ", lr)
        for kl in kl_weight:
            combination.append("layer: {} lr: {} kl: {}".format(layer,lr,kl))
            _ , _, _, train_accuracy, val_accuracy, _ = train_model_and_evaluate_classification(Xtrain_tensor, Ytrain_tensor, Xval_tensor, Yval_tensor, layer, learning_param = lr, kl_weight = kl, steps = 100, printStep = False)
            train_accuracy_list.append(train_accuracy)
            val_accuracy_list.append(val_accuracy)
            
print("Complete")

--- Layer:  [32, 16]
-- Learning Param:  0.001
-- Learning Param:  0.011
-- Learning Param:  0.021
-- Learning Param:  0.032
-- Learning Param:  0.042
-- Learning Param:  0.052
-- Learning Param:  0.062
-- Learning Param:  0.072
-- Learning Param:  0.082
-- Learning Param:  0.093
-- Learning Param:  0.103
-- Learning Param:  0.113
-- Learning Param:  0.123
-- Learning Param:  0.133
-- Learning Param:  0.144
-- Learning Param:  0.154
-- Learning Param:  0.164
-- Learning Param:  0.174
-- Learning Param:  0.184
-- Learning Param:  0.194
-- Learning Param:  0.205
-- Learning Param:  0.215
-- Learning Param:  0.225
-- Learning Param:  0.235
-- Learning Param:  0.245
-- Learning Param:  0.256
-- Learning Param:  0.266
-- Learning Param:  0.276
-- Learning Param:  0.286
-- Learning Param:  0.296
-- Learning Param:  0.307
-- Learning Param:  0.317
-- Learning Param:  0.327
-- Learning Param:  0.337
-- Learning Param:  0.347
-- Learning Param:  0.357
-- Learning Param:  0.368
-- Learning Param

In [63]:
results = pd.DataFrame({"Combination": combination,"Train Accuracy":train_accuracy_list, "Val Accuracy":val_accuracy_list})
results.to_csv("Combinations_classification_withoutSentiments&lagged2.csv")

## Find the hyperparameters with gives the highest val accuracy
results[results['Val Accuracy'] ==  results['Val Accuracy'].max()]

,Combination,Train Accuracy,Val Accuracy
258,"layer: [32, 16] lr: 0.052 kl: 0.082",0.643836,0.733333
452,"layer: [32, 16] lr: 0.093 kl: 0.021",0.753425,0.733333
612,"layer: [32, 16] lr: 0.123 kl: 0.123",0.657534,0.733333
754,"layer: [32, 16] lr: 0.154 kl: 0.042",0.602740,0.733333
805,"layer: [32, 16] lr: 0.164 kl: 0.052",0.547945,0.733333


In [71]:
layer = [32, 16]
lr = 0.093
kl = 0.021

_ , _, _, train_accuracy, val_accuracy, _ = train_model_and_evaluate_classification(Xtrain_tensor, Ytrain_tensor, Xval_tensor, Yval_tensor, layers = layer, learning_param = lr, kl_weight = kl, steps = 100, printStep = False)
print("Train Accuracy: ",train_accuracy)
print("Val Accuracy: ",val_accuracy)

Train Accuracy:  0.7534246575342466
Val Accuracy:  0.7333333333333333


**7. Retrain the model with selected hyperparameters and all train data available**

In [72]:
### Helper function for retraining the model with all data available
def train_model(Xtrain_tensor, Ytrain_tensor, layers = [100,20], learning_param = 0.01, kl_weight = 0.01, steps = 100, threshold = 0.50):    
    """ 
    Trains model and returns predictions on entire dataset.
    """
    
    in_features = Xtrain_tensor.shape[1]
    batch_size = Xtrain_tensor.shape[0]
    
    ## Ensure reproducibility
    seed = 1
    torch.manual_seed(seed)
    
    # Build model
    layer = []
    
    ## Input layer
    layer.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features = in_features, out_features = layers[0]))
    layer.append(nn.ReLU())
    
    ## Hidden layers
    for index, neurons in enumerate(layers):
        if index != (len(layers)-1):
            layer.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=neurons, out_features=layers[index+1]))
            layer.append(nn.ReLU())

    ## Output layer
    layer.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=layers[-1], out_features=1))
    layer.append(nn.Sigmoid())
    
    model = nn.Sequential(*layer)
    
    ## Print Model
    #print(summary(model,(batch_size,in_features)))
    #print(model)
    
    ### Define Loss - CrossEntropy for classification
    cross_entropy_loss = nn.BCELoss()
    kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)

    ## Define optimiser with learning rate
    optimizer = optim.Adam(model.parameters(), lr = learning_param)
    
    ### Train model
    for step in range(steps):
        torch.manual_seed(seed)
        
        pre = model(Xtrain_tensor)
        cross_entropy = cross_entropy_loss(pre, Ytrain_tensor.reshape(-1, 1).type(torch.FloatTensor))
        kl = kl_loss(model)
        total_cost = cross_entropy + kl_weight*kl

        optimizer.zero_grad()
        total_cost.backward()
        optimizer.step()
        
    pre_binary = pd.Series(pre.detach().numpy().flatten()).apply(lambda x: 1 if x >= threshold else 0)
    
    return (model, pre_binary)

In [73]:
### Standardise refit data (train + val)
Xrefit_tensor, _, Yrefit_tensor, _ = scale_and_convert_to_tensor(X_refit, X_refit, y_refit, y_refit)
print("X refit tensor",Xrefit_tensor.shape)
print("Y refit tensor",Yrefit_tensor.shape)

X refit tensor torch.Size([88, 9])
Y refit tensor torch.Size([88])


In [74]:
### Train on refit data and evaluate on test
y_predict_binary , _, _, train_accuracy, test_accuracy, _ = train_model_and_evaluate_classification(Xrefit_tensor, Yrefit_tensor, Xtest_tensor, Ytest_tensor, layer, learning_param = lr, kl_weight = kl, steps = 100, printStep = False)
print("Train Accuracy: ",train_accuracy)
print("Test Accuracy: ",test_accuracy)
print(y_predict_binary)

Train Accuracy:  0.6818181818181818
Test Accuracy:  0.5714285714285714
0     1
1     1
2     1
3     1
4     1
5     1
6     0
7     1
8     1
9     1
10    1
11    1
12    0
13    0
dtype: int64


**8. Obtain predictions for Backtesting**

In [75]:
### Standardise full data 
Xfull_tensor, _, Yfull_tensor, _ = scale_and_convert_to_tensor(X_full, X_full, y_full, y_full)
print("X test tensor",Xfull_tensor.shape)
print("Y test tensor",Yfull_tensor.shape)

X test tensor torch.Size([102, 9])
Y test tensor torch.Size([102])


In [76]:
model, pre_binary = train_model(Xfull_tensor, Yfull_tensor, layers = layer, learning_param = lr, kl_weight = kl, steps = 100)

In [77]:
accuracy_score(y_full, pre_binary)

0.6176470588235294

In [80]:
#pd.DataFrame(pre_binary).to_csv("out.csv")

## With Sentiments

**1. Create Lags**

In [81]:
feature_lags = {"Adj_Close_BTC-USD" : 1, 
                "Adj_Close_SPY" : 1,
                "Adj_Close_GLD" : 1,
                "Adj_Close_CHFUSD=X" : 1,
                "Adj_Close_EURUSD=X" : 1,
                "Adj_Close_GBPUSD=X" : 1,
                "Adj_Close_JPYUSD=X" : 1,
                "blockchain_transactions_per_block" : 1,
                "blockchain_hash_rates" : 1,
                "coindesk_sentiment" : 1,
                "reddit_comments_sentiments" : 1,
                "top_50_reddit_posts_sentiments" : 1}


data = lag(df, feature_lags)

**2. Handle Train-test split**

In [82]:
data = data[(data["date"] >= "2021-01-01") & (data["date"] <= "2021-04-12")]

## train, validation and test split
train = data[(data["date"] >= "2021-01-01") & (data["date"] <= "2021-03-14")]
validation = data[(data["date"] >= "2021-03-15") & (data["date"] <= "2021-03-29")]
test = data[(data["date"] >= "2021-03-30") & (data["date"] <= "2021-04-12")]

## refit (refit = train + validation) and full for later use 
refit = data[(data["date"] >= "2021-01-01") & (data["date"] <= "2021-03-29")]
full = data.copy(deep = True)

## train 
X_train = train.drop(["date", "class_y"], axis = 1)
y_train = train["class_y"]

## val
X_val = validation.drop(["date", "class_y"], axis = 1)
y_val = validation["class_y"]

## test
X_test = test.drop(["date", "class_y"], axis = 1)
y_test = test["class_y"]

## refit
X_refit = refit.drop(["date", "class_y"], axis = 1)
y_refit = refit["class_y"]

## full
X_full = full.drop(["date", "class_y"], axis = 1)
y_full = full["class_y"]

**3. Standardise dataset and transform to tensors**

In [83]:
## Standardise datasets and convert into tensors
Xtrain_tensor, Xval_tensor, Ytrain_tensor, Yval_tensor = scale_and_convert_to_tensor(X_train, X_val, y_train, y_val)
_, Xtest_tensor, _, Ytest_tensor = scale_and_convert_to_tensor(X_train, X_test, y_train, y_test)

print("X train tensor",Xtrain_tensor.shape)
print("Y train tensor",Ytrain_tensor.shape)

print("X val tensor",Xval_tensor.shape)
print("Y val tensor",Yval_tensor.shape)

print("X test tensor",Xtest_tensor.shape)
print("Y test tensor",Ytest_tensor.shape)

X train tensor torch.Size([73, 12])
Y train tensor torch.Size([73])
X val tensor torch.Size([15, 12])
Y val tensor torch.Size([15])
X test tensor torch.Size([14, 12])
Y test tensor torch.Size([14])


**4. Perform Gridsearch**

In [84]:
### Hyperparameter Tuning --> find best parameters
train_accuracy_list = []
val_accuracy_list = []
combination = []

learning_param_list = pd.Series(np.linspace(0.001,0.5,50)).apply(lambda x: round(x,3))
kl_weight =  pd.Series(np.linspace(0.001,0.5,50)).apply(lambda x: round(x,3))
layers_list = ([32,16],[32,8],[32,16,8])

for layer in layers_list:
    print("--- Layer: ", layer)
    for lr in learning_param_list:
        print("-- Learning Param: ", lr)
        for kl in kl_weight:
            combination.append("layer: {} lr: {} kl: {}".format(layer,lr,kl))
            _ , _, _, train_accuracy, val_accuracy, _ = train_model_and_evaluate_classification(Xtrain_tensor, Ytrain_tensor, Xval_tensor, Yval_tensor, layer, learning_param = lr, kl_weight = kl, steps = 100, printStep = False)
            train_accuracy_list.append(train_accuracy)
            val_accuracy_list.append(val_accuracy)
            
print("Complete")

--- Layer:  [32, 16]
-- Learning Param:  0.001
-- Learning Param:  0.011
-- Learning Param:  0.021
-- Learning Param:  0.032
-- Learning Param:  0.042
-- Learning Param:  0.052
-- Learning Param:  0.062
-- Learning Param:  0.072
-- Learning Param:  0.082
-- Learning Param:  0.093
-- Learning Param:  0.103
-- Learning Param:  0.113
-- Learning Param:  0.123
-- Learning Param:  0.133
-- Learning Param:  0.144
-- Learning Param:  0.154
-- Learning Param:  0.164
-- Learning Param:  0.174
-- Learning Param:  0.184
-- Learning Param:  0.194
-- Learning Param:  0.205
-- Learning Param:  0.215
-- Learning Param:  0.225
-- Learning Param:  0.235
-- Learning Param:  0.245
-- Learning Param:  0.256
-- Learning Param:  0.266
-- Learning Param:  0.276
-- Learning Param:  0.286
-- Learning Param:  0.296
-- Learning Param:  0.307
-- Learning Param:  0.317
-- Learning Param:  0.327
-- Learning Param:  0.337
-- Learning Param:  0.347
-- Learning Param:  0.357
-- Learning Param:  0.368
-- Learning Param

In [85]:
results = pd.DataFrame({"Combination": combination,"Train Accuracy":train_accuracy_list, "Val Accuracy":val_accuracy_list})
results.to_csv("Combinations_classification_withSentiments&lagged2.csv")

## Find the hyperparameters with gives the highest val and train accuracy
results[results['Val Accuracy'] ==  results['Val Accuracy'].max()]

,Combination,Train Accuracy,Val Accuracy
2705,"layer: [32, 8] lr: 0.042 kl: 0.052",0.917808,0.866667
3639,"layer: [32, 8] lr: 0.225 kl: 0.398",0.547945,0.866667


In [86]:
layer = [32, 8]
lr = 0.042
kl =0.052

_ , _, _, train_accuracy, val_accuracy, _ = train_model_and_evaluate_classification(Xtrain_tensor, Ytrain_tensor, Xval_tensor, Yval_tensor, layer, learning_param = lr, kl_weight = kl, steps = 100, printStep = False)
print("Train Accuracy: ",train_accuracy)
print("Val Accuracy: ",val_accuracy)

Train Accuracy:  0.9178082191780822
Val Accuracy:  0.8666666666666667


**5. Retrain the model with selected hyperparameters and all data available**

In [87]:
### Standardise refit data (train + val)
Xrefit_tensor, _, Yrefit_tensor, _ = scale_and_convert_to_tensor(X_refit, X_refit, y_refit, y_refit)

print("X refit tensor",Xrefit_tensor.shape)
print("Y refit tensor",Yrefit_tensor.shape)

X refit tensor torch.Size([88, 12])
Y refit tensor torch.Size([88])


In [88]:
### Train on refit data and evaluate on test
y_predict_binary , _, _, train_accuracy, test_accuracy, _ = train_model_and_evaluate_classification(Xrefit_tensor, Yrefit_tensor, Xtest_tensor, Ytest_tensor, layer, learning_param = lr, kl_weight = kl, steps = 100, printStep = False)
print("Train Accuracy: ",train_accuracy)
print("Test Accuracy: ",test_accuracy)
print(y_predict_binary)

Train Accuracy:  0.8977272727272727
Test Accuracy:  0.6428571428571429
0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
dtype: int64


**6. Obtain predictions for Backtesting**

In [89]:
### Standardise full data 
Xfull_tensor, _, Yfull_tensor, _ = scale_and_convert_to_tensor(X_full, X_full, y_full, y_full)
print("X test tensor",Xfull_tensor.shape)
print("Y test tensor",Yfull_tensor.shape)

X test tensor torch.Size([102, 12])
Y test tensor torch.Size([102])


In [90]:
model, pre_binary = train_model(Xfull_tensor, Yfull_tensor, layers = layer, learning_param = lr, kl_weight = kl, steps = 100)
accuracy_score(y_full, pre_binary)

0.8823529411764706

In [92]:
#pd.DataFrame(pre_binary).to_csv("out.csv")